In [16]:
import cv2, numpy as np, os


def detect_gaps(img_path):
    img = cv2.imread(img_path, 0)
    _, bin_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    skel = np.zeros(bin_img.shape, np.uint8)
    temp = np.zeros(bin_img.shape, np.uint8)
    eroded = np.zeros(bin_img.shape, np.uint8)
    elem = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    done = False
    while not done:
        eroded = cv2.erode(bin_img, elem)
        temp = cv2.dilate(eroded, elem)
        temp = cv2.subtract(bin_img, temp)
        cv2.bitwise_or(skel, temp, skel)
        bin_img = eroded.copy()
        if cv2.countNonZero(bin_img) == 0:
            done = True

    gap_points = []
    rows, cols = skel.shape
    for r in range(1, rows - 1):
        for c in range(1, cols - 1):
            if skel[r, c] == 255:
                area = skel[r - 1 : r + 2, c - 1 : c + 2]
                if cv2.countNonZero(area) == 2:
                    gap_points.append((c, r))

    out = cv2.imread(img_path)
    for i in range(len(gap_points) - 1):
        x1, y1 = gap_points[i]
        x2, y2 = gap_points[i + 1]
        dist = np.hypot(x2 - x1, y2 - y1)
        if dist < 50:
            mx = (x1 + x2) // 2
            my = (y1 + y2) // 2
            cv2.circle(out, (mx, my), 10, (0, 0, 255), 3)

    cv2.imshow("Hasil Gap Detection", out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


detect_gaps(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\static\images\test\disconnected.jpg")

In [ ]:
import os
script_dir = os.path.dirname(os.getcwd())
print(os.path.dirname(script_dir))


# c detection (vid)

In [6]:
import cv2
import numpy as np


def detect_c_breaks_bbox_frame(frame, gap_threshold=20):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_img, connectivity=8)
    bboxes = []
    for label_id in range(1, num_labels):
        x, y, w, h, area = stats[label_id]
        if area > 10:
            bboxes.append((x, y, x + w, y + h))
    if len(bboxes) <= 1:
        return frame, False
    bboxes.sort(key=lambda b: b[1])
    putus_detected = False
    for i in range(len(bboxes) - 1):
        lA, tA, rA, bA = bboxes[i]
        lB, tB, rB, bB = bboxes[i + 1]
        distance = tB - bA
        if distance > gap_threshold:
            putus_detected = True
            mxA = (lA + rA) // 2
            mxB = (lB + rB) // 2
            mx_gap = (mxA + mxB) // 2
            my_gap = bA + distance // 2
            cv2.circle(frame, (mx_gap, my_gap), 10, (0, 0, 255), 3)
    return frame, putus_detected


def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Webcam not detected")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result_frame, is_broken = detect_c_breaks_bbox_frame(frame, gap_threshold=30)
        if is_broken:
            cv2.putText(result_frame, "WARNING: Ada Huruf C Terputus", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(result_frame, "PASS: Tidak Ada Gap Terputus", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("C Break Detection", result_frame)
        if cv2.waitKey(1) & 0xFF == 27:  # tekan ESC
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

# c detection (img)

In [ ]:
import cv2
import numpy as np


def detect_c_breaks_bbox(img_path, gap_threshold=20):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Jika huruf lebih gelap, pakai THRESH_BINARY, sesuaikan
    _, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Cari connected components
    # stats: [x, y, width, height, area]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_img, connectivity=8)

    # label 0 = background, sisanya objek
    bboxes = []
    for label_id in range(1, num_labels):
        x, y, w, h, area = stats[label_id]
        if area > 10:
            bboxes.append((x, y, x + w, y + h))  # (left, top, right, bottom)

    if len(bboxes) <= 1:
        print("Tidak terdeteksi lebih dari 1 huruf C, dianggap PASS.")
        cv2.imshow("Result", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    # Urutkan berdasarkan top (y)
    bboxes.sort(key=lambda b: b[1])

    # Periksa gap di antara bounding box berurutan
    # bottom dari box atas = b[3], top dari box bawah = b'[1]
    putus_detected = False
    for i in range(len(bboxes) - 1):
        leftA, topA, rightA, bottomA = bboxes[i]
        leftB, topB, rightB, bottomB = bboxes[i + 1]

        distance = topB - bottomA
        if distance > gap_threshold:
            putus_detected = True
            # Tentukan titik tengah gap
            # X ambil rata-rata tengah bounding box A dan B
            midX_A = (leftA + rightA) // 2
            midX_B = (leftB + rightB) // 2
            midX_gap = (midX_A + midX_B) // 2

            # Y di tengah jarak antara bottomA dan topB
            midY_gap = bottomA + distance // 2

            cv2.circle(img, (midX_gap, midY_gap), 10, (0, 0, 255), 3)

    if putus_detected:
        print("WARNING: Ada huruf C terputus. Lingkaran merah menunjukkan lokasi gap.")
    else:
        print("PASS: Tidak ada gap berarti di antara huruf C.")
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


detect_c_breaks_bbox(r"C:\xampp\htdocs\VISUALAI\website-django\inspection\static\images\test\disconnected.jpg", gap_threshold=30)